<a href="https://colab.research.google.com/github/weasel-codes/anomaly-detection-iot/blob/main/anomaly_detection_preprocess2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
import pandas as pd
import numpy as np
import glob

In [15]:
base_url = '/content/drive/MyDrive/ML/anomaly/'
data_file = 'dataset.csv'

In [16]:
df = pd.read_csv(base_url+data_file)
if df.shape[1] > 8 :
  df = df.drop(columns=df.columns[0])

df

,frame.number,frame.time_relative,frame.len,data.len,ip.src,ip.dst,frame.protocols,label
0,88211,70.949809,1514,NaN,104.74.213.186,192.168.0.24,eth:ethertype:ip:tcp,0
1,58708,132.842376,54,NaN,192.168.0.19,173.194.49.203,eth:ethertype:ip:tcp,0
2,131894,296.013317,66,NaN,192.168.0.16,192.168.0.13,eth:ethertype:ip:tcp,0
3,252744,49.393385,74,32.0,192.168.0.24,210.89.164.90,eth:ethertype:ip:udp:data,1
4,35069,66.823389,1502,NaN,192.168.0.13,192.168.0.16,eth:ethertype:ip:tcp,0
...,...,...,...,...,...,...,...,...
2722818,91235,152.817776,77,35.0,192.168.0.24,210.89.164.90,eth:ethertype:ip:udp:data,1
2722819,137817,59.616331,74,32.0,192.168.0.13,210.89.164.90,eth:ethertype:ip:udp:data,1
2722820,18178,59.422203,1498,NaN,192.168.0.13,192.168.0.16,eth:ethertype:ip:tcp,0
2722821,77956,285.853238,188,NaN,192.168.0.24,223.39.123.191,eth:ethertype:ip:tcp:ssl,0


In [17]:
df.isna().sum()

frame.number                 0
frame.time_relative          0
frame.len                    0
data.len               1441202
ip.src                   12764
ip.dst                   12764
frame.protocols              0
label                        0
dtype: int64

In [18]:
df['data.len'] = df['data.len'].fillna(0)
df['ip.src'] = df['ip.src'].fillna(method='ffill')
df['ip.dst'] = df['ip.dst'].fillna(method='ffill')
df.isna().sum()

frame.number           0
frame.time_relative    0
frame.len              0
data.len               0
ip.src                 0
ip.dst                 0
frame.protocols        0
label                  0
dtype: int64

In [19]:
df

,frame.number,frame.time_relative,frame.len,data.len,ip.src,ip.dst,frame.protocols,label
0,88211,70.949809,1514,0.0,104.74.213.186,192.168.0.24,eth:ethertype:ip:tcp,0
1,58708,132.842376,54,0.0,192.168.0.19,173.194.49.203,eth:ethertype:ip:tcp,0
2,131894,296.013317,66,0.0,192.168.0.16,192.168.0.13,eth:ethertype:ip:tcp,0
3,252744,49.393385,74,32.0,192.168.0.24,210.89.164.90,eth:ethertype:ip:udp:data,1
4,35069,66.823389,1502,0.0,192.168.0.13,192.168.0.16,eth:ethertype:ip:tcp,0
...,...,...,...,...,...,...,...,...
2722818,91235,152.817776,77,35.0,192.168.0.24,210.89.164.90,eth:ethertype:ip:udp:data,1
2722819,137817,59.616331,74,32.0,192.168.0.13,210.89.164.90,eth:ethertype:ip:udp:data,1
2722820,18178,59.422203,1498,0.0,192.168.0.13,192.168.0.16,eth:ethertype:ip:tcp,0
2722821,77956,285.853238,188,0.0,192.168.0.24,223.39.123.191,eth:ethertype:ip:tcp:ssl,0


In [21]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['ip.src']= label_encoder.fit_transform(df['ip.src'])
df['ip.dst']= label_encoder.fit_transform(df['ip.dst'])
df[:5]

,frame.number,frame.time_relative,frame.len,data.len,ip.src,ip.dst,frame.protocols,label
0,88211,70.949809,1514,0.0,56,27002,eth:ethertype:ip:tcp,0
1,58708,132.842376,54,0.0,26921,26821,eth:ethertype:ip:tcp,0
2,131894,296.013317,66,0.0,26913,26966,eth:ethertype:ip:tcp,0
3,252744,49.393385,74,32.0,26935,27350,eth:ethertype:ip:udp:data,1
4,35069,66.823389,1502,0.0,26905,26977,eth:ethertype:ip:tcp,0


In [23]:
from sklearn.preprocessing import MinMaxScaler
  
# Creating an instance of the sklearn.preprocessing.MinMaxScaler()
scaler = MinMaxScaler()
  
# Scaling the Price column of the created dataFrame and storing
# the result in ScaledPrice Column
df[['frame.time_relative']] = scaler.fit_transform(df[['frame.time_relative']])
df[:5]

,frame.number,frame.time_relative,frame.len,data.len,ip.src,ip.dst,frame.protocols,label
0,88211,0.219518,1514,0.0,56,27002,eth:ethertype:ip:tcp,0
1,58708,0.411014,54,0.0,26921,26821,eth:ethertype:ip:tcp,0
2,131894,0.915864,66,0.0,26913,26966,eth:ethertype:ip:tcp,0
3,252744,0.152823,74,32.0,26935,27350,eth:ethertype:ip:udp:data,1
4,35069,0.206751,1502,0.0,26905,26977,eth:ethertype:ip:tcp,0


In [9]:
# Create Entites using indexes of csv
X = df.iloc[:, :-1].values #All the rows + all the columns except last
Y = df.iloc[:,-1] #Dependent vairables is last column
X.shape, Y.shape

((2722823, 7), (2722823,))

In [71]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

TypeError: ignored

In [ ]:
X[:,5]

In [ ]:
df['frame.protocols'].unique()

In [ ]:
for i in range(df.shape[0]):
  var = df['frame.protocols'][i].split(':')
  print(var)
  temp = ""
  if len(var) == 1 :
    temp = ""
  elif len(var) == 2 :
    temp = temp + var[0] + ":" + var[1]
  elif len(var) == 3 :
    temp = temp + var[0] + ":" + var[1] + ":" + var[2]
  elif len(var) == 4 :
    temp = temp + var[0] + ":" + var[1] + ":" + var[2] + ":" + var[3]

  df['frame.protocols'][i] = temp  
  print(df['frame.protocols'][i])
  break